In [18]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import json

In [3]:
base_url = 'https://murmansk.hh.ru/'
headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
           "Accept": "*/*"}
params = {'text': 'pharma'}
url = f'{base_url}/search/vacancy'

In [4]:
response = requests.get(url, headers=headers, params=params)

In [5]:
dom = BeautifulSoup(response.text, 'html.parser')

In [6]:
vacancy = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})

In [7]:
vacancy_list = []

In [19]:
for items in vacancy:
  vacancy_data = {}

  info = items.find('a', {'class': 'serp-item__title'})
  link = base_url + info['href']
  name = info.getText()
  company = items.find('a', {'class': 'bloko-link_kind-tertiary'}).getText()
  views = items.find('div', {'class': 'online-users'})
  
  try:
    salary = items.find('span', {'class': 'bloko-header-section-3'})
  
    if len(salary.split()) == 6:
      min_salary = salary.split()[0] + salary.split()[1]
      max_salary = salary.split()[3] + salary.split()[4]
      valute = salary.split()[5]

    elif len(salary.split()) == 4 and salary.split()[0] == 'от':
      min_salary = salary.split()[1] + salary.split()[2]
      max_salary = None
      valute = salary.split()[3]


    else:
      min_salary = None
      max_salary = salary.split()[1] + salary.split()[2]
      valute = salary.split()[3]
  except:
    min_salary = None
    max_salary = None
    valute = None

  vacancy_data['name'] = name
  vacancy_data['link'] = link
  vacancy_data['company'] = company
  vacancy_data['views'] = views
  try:
    vacancy_data['min_salary'] = int(min_salary)
    vacancy_data['max_salary'] = int(max_salary)
    vacancy_data['valute'] = valute
  except:
    vacancy_data['min_salary'] = min_salary
    vacancy_data['max_salary'] = max_salary
    vacancy_data['valute'] = valute

  vacancy_list.append(vacancy_data)

with open('job.json', 'w', encoding='utf-8') as f:
    json.dump(vacancy_list, f, ensure_ascii=False)
